# BDA ANALYSEPROJEKT MIETPREISE DEUTSCHLAND 2023


## Betriebswirtschaftliche Datenanalyse – Exposé

### Teammitglieder

| Vorname   | Nachname  | Matrikelnummer |
|-----------|-----------|----------------|
| Timon     | Schmidt   | 217101         |
| Sümeyye   | Gügercin  | 214424         |
| Melih     | Akbulut   | 220860         |
| Dariana   | Barkov    | 220858         |
| Rasmus    | Wegat     | 221727         |

---

### Fragestellung

**„Gibt es einen klaren Zusammenhang zwischen Leerstand und Mietpreisen in Deutschland?“**

---

### Erläuterung

In dieser Analyse sollen die Daten zu leerstehenden Wohnungen und den Wohnungspreisen in den jeweiligen Landkreisen aus dem Jahr 2023 verglichen werden. Ziel ist es, zu untersuchen, ob in attraktiven Städten mit hohen Mieten ein künstlicher Wohnungsleerstand existiert, um durch eine künstliche Verknappung die Preise hochzuhalten. Darüber hinaus sollen weitere Muster identifiziert werden, um die Haupttreiber hoher Mietpreise in Großstädten besser zu verstehen.

---

### Ansatz und Vorgehensweise

Mit Hilfe von **pandas**, **matplotlib** und **seaborn** werden die Leerstandsquoten und Mietpreise auf Kreisebene anhand der Daten des Deutschlandatlas analysiert und visualisiert. Zusätzlich können weitere Einflussfaktoren wie Bevölkerungsdichte, Einkommen und Urbanität einbezogen werden, um weitere Aussagen über Ursachen hoher Mietpreise treffen zu können.

---

### Datenquelle

[Deutschlandatlas – Mieten](https://www.deutschlandatlas.bund.de/DE/Karten/Wie-wir-wohnen/040-Mieten.html#_g5f1sd6f4)  
Aufgerufen am 13.04.2025, 13:00 Uhr.

Imports

In [3]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import seaborn as sns
from scipy.stats import lognorm, kstest, norm, shapiro

%matplotlib inline

Einlesen der Exel Datei und herausfiltern der Daten die wir wollen


In [ ]:

# Tabellen laden
gem1222 = pd.read_excel("Deutschlandatlas-Daten.xlsx", sheet_name="Deutschlandatlas_GEM1222")
vbgem1222 = pd.read_excel("Deutschlandatlas-Daten.xlsx", sheet_name="Deutschlandatlas_VBGEM1222")
krs1222 = pd.read_excel("Deutschlandatlas-Daten.xlsx", sheet_name="Deutschlandatlas_KRS1222")
krs1221 = pd.read_excel("Deutschlandatlas-Daten.xlsx", sheet_name="Deutschlandatlas_KRS1221")
krs1220 = pd.read_excel("Deutschlandatlas-Daten.xlsx", sheet_name="Deutschlandatlas_KRS1220")

# Einheitlicher Key-Name
def rename_key(df):
    return df.rename(columns={df.columns[0]: "KRS_ID"})

gem1222 = rename_key(gem1222)
vbgem1222 = rename_key(vbgem1222)
krs1222 = rename_key(krs1222)
krs1221 = rename_key(krs1221)
krs1220 = rename_key(krs1220)

# ❗️Auswahl der gewünschten Spalten je Ebene (inkl. Name-Spalten)
gem_cols = ["KRS_ID", "Gemeindename", "bev_dicht", "bev_entw", "beschq_insg"]
vbgem_cols = ["KRS_ID", "Gemeindeverbandsname", "pendel"]
krs1222_cols = ["KRS_ID", "Kreisname", "bev_binw", "preis_miet", "wohn_eigen", "wohn_leer", "wohn_EZFH", "wohn_MFH",
                "erw_wachs", "erw_mini", "erw_minineben", "alq", "schulden", "sozsich", "straft", "einbr"]
krs1221_cols = ["KRS_ID", "Kreisname", "preis_baul", "erw_bip", "hh_veink"]
krs1220_cols = ["KRS_ID", "Kreisname", "erw_vol"]

# ❗️Nur benötigte Spalten extrahieren
gem1222_sel = gem1222[gem_cols]
vbgem1222_sel = vbgem1222[vbgem_cols]
krs1222_sel = krs1222[krs1222_cols]
krs1221_sel = krs1221[krs1221_cols]
krs1220_sel = krs1220[krs1220_cols]

# Kombinieren aller Tabellen über KRS_ID
# Reihenfolge: Kreisebene (groß), dann Verbandsgemeinden, dann Gemeinden
df = krs1222_sel.merge(krs1221_sel, on=["KRS_ID", "Kreisname"], how="left", suffixes=("", "_krs1221"))
df = df.merge(krs1220_sel, on=["KRS_ID", "Kreisname"], how="left", suffixes=("", "_krs1220"))
df = df.merge(vbgem1222_sel, on="KRS_ID", how="left")
df = df.merge(gem1222_sel, on="KRS_ID", how="left")

# Nach KRS_ID sortieren
df = df.sort_values("KRS_ID").reset_index(drop=True)

# Optional: Speichern
df.to_excel("ExtractedData.xlsx", index=False)


    KRS_ID               Kreisname  bev_binw             preis_miet  \
0  1001000        Flensburg, Stadt     49.81   8,50 bis unter 10,00   
1  1002000  Kiel, Landeshauptstadt    -12.80   8,50 bis unter 10,00   
2  1003000      Lübeck, Hansestadt     30.67  10,00 bis unter 11,50   
3  1004000       Neumünster, Stadt   -286.79    7,00 bis unter 8,50   
4  1051000            Dithmarschen     66.47    7,00 bis unter 8,50   

   wohn_eigen  wohn_leer  wohn_EZFH  wohn_MFH  erw_wachs  erw_mini  ...  \
0   24.471627       2.95        4.1      21.6       0.96     10.06  ...   
1   24.250576       2.13        1.4      18.6       1.07      9.51  ...   
2   28.982434       2.88        6.1      32.1       0.78      8.56  ...   
3   36.995588       3.71        5.9      22.8       1.67      8.77  ...   
4   57.855063       3.49       27.2      17.4       1.02     12.13  ...   

          preis_baul  erw_bip  hh_veink  erw_vol  Gemeindeverbandsname  \
0  100 bis unter 200    65.27     20.15     1.33

**Legende der Kombinierten Tabelle**

- **bev_dicht**: Bevölkerungsdichte (Einwohner je km²)
- **bev_entw**: Bevölkerungsentwicklung (% zum Vorjahr)
- **beschq_insg**: Beschäftigungsquote insgesamt (%)
- **pendel**: Pendleranteil bzw. Pendlerbewegung
- **bev_binw**: Bevölkerungsentwicklung insgesamt (%-Veränderung)
- **preis_miet**: Angebotsmietpreis in €/m²
- **wohn_eigen**: Anteil der selbstgenutzten Immobilien (%)
- **wohn_leer**: Leerstandsquote (%)
- **wohn_EZFH**: Anteil an Ein- und Zweifamilienhäusern (%)
- **wohn_MFH**: Anteil an Mehrfamilienhäusern (%)
- **erw_wachs**: Erwerbstätigenwachstum (%)
- **erw_mini**: Anteil der Minijobs (%)
- **erw_minineben**: Anteil der Minijobs als Nebentätigkeit (%)
- **alq**: Arbeitslosenquote (%)
- **schulden**: Verschuldung je Einwohner (in €)
- **sozsich**: Anteil an der Bevölkerung in sozialer Sicherung (%)
- **straft**: Straftaten pro 100.000 Einwohner
- **einbr**: Einbruchshäufigkeit pro 100.000 Einwohner
- **preis_baul**: Baulandpreise (Textkategorie, z. B. „100 bis unter 200 €/m²“)
- **erw_bip**: Bruttoinlandsprodukt (BIP) je Erwerbstätigen (in Tsd. €)
- **hh_veink**: Verfügbares Einkommen je Einwohner (in Tsd. €)
- **erw_vol**: Anteil der Vollzeit-Erwerbstätigen an allen Erwerbstätigen (%)